In [29]:
import sqlite3
import pandas as pd
import requests
from bs4 import BeautifulSoup

Task
----------
Design and create a database for scraping All books for each category using DB Browser for sqllite from this web site (http://books.toscrape.com/)<br/>
1- create books & categories table<br/>
2- create notebook to scrap data and save data in the database

#### Create Tables

In [32]:
connection_obj = sqlite3.connect('task.db')
 
cursor_obj = connection_obj.cursor()
 
cursor_obj.execute("DROP TABLE IF EXISTS CATEGORY")
table = """ CREATE TABLE CATEGORY (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255) NOT NULL
        );"""
cursor_obj.execute(table)

cursor_obj.execute("DROP TABLE IF EXISTS BOOK")
table = """ CREATE TABLE BOOK (
            id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255) NOT NULL,
            price FLOAT,
            rate INT,
            categ_id INT,
            FOREIGN KEY (categ_id) REFERENCES CATEGORY(id)
        ); """
 
cursor_obj.execute(table)
print("Tables is Ready")
connection_obj.close()

Tables is Ready


#### Add Categories & Books

In [33]:
connection_obj = sqlite3.connect('task.db')
cursor_obj = connection_obj.cursor()

rate_keys={'One':1, 'Two':2, 'Three':3, 'Four':4, 'Five':5}

def add_category(categ_name):
    cursor_obj.execute('insert into category (name) values (?)',[categ_name])
    categ_id = cursor_obj.lastrowid
    connection_obj.commit()
    return categ_id
        
def add_books(url,categ_href,categ_id):
    categ_soup=BeautifulSoup(requests.get(str(url+categ_href)).text,'html.parser')
    books=categ_soup.find_all('article',attrs={'class':'product_pod'})
    for book in books:
        name=book.find('h3').find('a').get('title')
        price=book.find('p', attrs={'class':'price_color'}).get_text()[2:]
        rate=rate_keys[str(book.find('p', attrs={'class':'star-rating'}).get('class')[1])]
        cursor_obj.execute('insert into book (name,price,rate,categ_id) values (?,?,?,?)',
                               [name,price,rate,int(categ_id)])
        connection_obj.commit()
    if categ_soup.find('li', attrs={'class':'next'}):
        next_link = categ_soup.find('li', attrs={'class':'next'}).find('a').get('href')
        url = str(url+categ_href.replace('index.html', next_link))
        add_books(url,categ_href,categ_id)
    
def add_category_books():
    url='https://books.toscrape.com/'
    soup=BeautifulSoup(requests.get(url).text,'html.parser')
    categ_lst=soup.find('ul',attrs={'class':''}).find_all('a')
    for categ in categ_lst:
        categ_name=categ.get_text().strip()
        categ_href=categ.get('href')
        categ_id=add_category(categ_name)
        add_books(url,categ_href,categ_id)

        
add_category_books()
connection_obj.close()